<a href="https://colab.research.google.com/github/juanprida/nlp_with_transformers/blob/main/08_making_transformers_efficient_in_production.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install transformers
!pip install datasets

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.3/6.3 MB 48.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.6/7.6 MB 61.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 199.2/199.2 KB 8.1 MB/s eta 0:00:00
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 469.0/469.0 KB 35.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.0/1.0 MB 64.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 212.2/212.2 KB 27.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 110.5/110.5 KB 15.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 132.9/132.9 KB 4.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 264.6/264.6 KB 33.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 158.8/158.8 KB 19.3 MB/s eta 0:00:0

In [23]:
from transformers import pipeline, TrainingArguments, Trainer, AutoTokenizer, AutoConfig, AutoModelForSequenceClassification

from datasets import load_metric
from datasets import load_dataset

from huggingface_hub import notebook_login

import torch
import torch.nn as nn
import torch.nn.functional as F

import numpy as np

from pathlib import Path
from time import perf_counter

In [24]:
notebook_login()

Token is valid.
Your token has been saved in your configured git credential helpers (store).
Your token has been saved to /root/.cache/huggingface/token
Login successful


In [3]:
bert_ckpt = "transformersbook/bert-base-uncased-finetuned-clinc"
pipe = pipeline("text-classification", model=bert_ckpt)

In [4]:
query = """Hey, I'd like to rent a vehicle from Nov 1st to Nov 15th in Paris and I need a 15 passenger van"""
pipe(query)

[{'label': 'car_rental', 'score': 0.549003541469574}]

In [5]:
clinc = load_dataset("clinc_oos", "plus")

Generating train split:   0%|          | 0/15250 [00:00<?, ? examples/s]

Generating validation split:   0%|          | 0/3100 [00:00<?, ? examples/s]

Generating test split:   0%|          | 0/5500 [00:00<?, ? examples/s]

Dataset clinc_oos downloaded and prepared to /root/.cache/huggingface/datasets/clinc_oos/plus/1.0.0/abcc41d382f8137f039adc747af44714941e8196e845dfbdd8ae7a7e020e6ba1. Subsequent calls will reuse this data.


  0%|          | 0/3 [00:00<?, ?it/s]

In [6]:
sample = clinc["test"][42]
sample

{'text': 'transfer $100 from my checking to saving account', 'intent': 133}

In [7]:
intents = clinc["train"].features["intent"]
intents.int2str(sample["intent"])

'transfer'

In [10]:
class PerformanceBenchmark:
    def __init__(self, pipeline, dataset, optim_type="BERT baseline"):
        self.pipeline = pipeline
        self.dataset = dataset
        self.optim_type = optim_type
        self.accuracy_score = load_metric("accuracy")

    def compute_accuracy(self):
        preds, labels = [], []
        for example in self.dataset:
            pred = self.pipeline(example["text"])[0]["label"]
            label = example["intent"]
            preds.append(intents.str2int(pred))
            labels.append(label)
        accuracy = self.accuracy_score.compute(predictions=preds, references=labels)
        print(f"Accuracy on test set - {accuracy['accuracy']:.3f}")
        return accuracy

    def compute_size(self):
        state_dict = self.pipeline.model.state_dict()
        tmp_path = Path("model.pt")
        torch.save(state_dict, tmp_path)
        # Calculate size in megabytes.
        size_mb = Path(tmp_path).stat().st_size / (1024 * 1024)
        # Delete temp file
        tmp_path.unlink()
        print(f"Model size (MB) is {size_mb:.2f}")
        return {"size_mb": size_mb}
    
    def time_pipeline(self, query="What is the pin for my account?"):
        latencies = []
        for _ in range(10):
            _ = self.pipeline(query)
        for _ in range(10):
            start_time = perf_counter()
            _ = self.pipeline(query)
            latency = perf_counter() - start_time
            latencies.append(latency)
        
        time_avg_ms = 1000 * np.mean(latencies)
        time_std_ms = 1000 * np.std(latencies)
        print(f"avg latency: {time_avg_ms:.2f} +/- {time_std_ms:.2f}")
        return {"time_avg_ms": time_avg_ms, "time_std_ms": time_std_ms}
    
    def run_benchmark(self):
        metrics = {}
        metrics[self.optim_type] = self.compute_size()
        metrics[self.optim_type].update(self.time_pipeline())
        metrics[self.optim_type].update(self.compute_accuracy())
        return metrics

In [11]:
pb = PerformanceBenchmark(pipe, clinc["test"])
perf_metrics = pb.run_benchmark()

Model size (MB) is 418.15
avg latency: 113.33 +/- 5.76
Accuracy on test set - 0.867


In [12]:
class DistillationTrainingArguments(TrainingArguments):
    def __init__(self, *args, alpha=0.5, temperature=2.0, **kwargs):
        super().__init__(*args, **kwargs)
        self.alpha = alpha
        self.temperature = temperature

In [14]:
class DistillationTrainer(Trainer):
    def __init__(self, *args, teacher_model=None, **kwargs):
        super().__init__(*args, **kwargs)
        self.teacher_model = teacher_model
        def compute_loss(self, model, inputs, return_outputs=False):
            outputs_stu = model(**inputs)
            
            # Extract cross-entropy loss and logits from student
            loss_ce = outputs_stu.loss
            logits_stu = outputs_stu.logits
            
            # Extract logits from teacher
            with torch.no_grad():
                outputs_tea = self.teacher_model(**inputs)
                logits_tea = outputs_tea.logits
            
            # Soften probabilities and compute distillation loss
            loss_fct = nn.KLDivLoss(reduction="batchmean")
            loss_kd = self.args.temperature ** 2 * loss_fct(
                F.log_softmax(logits_stu / self.args.temperature, dim=-1),
                F.softmax(logits_tea / self.args.temperature, dim=-1))
            
            # Return weighted student loss
            loss = self.args.alpha * loss_ce + (1. - self.args.alpha) * loss_kd
            return (loss, outputs_stu) if return_outputs else loss

In [15]:
student_ckpt = "distilbert-base-uncased"
student_tokenizer = AutoTokenizer.from_pretrained(student_ckpt)

def tokenize_text(batch):
    return student_tokenizer(batch["text"], truncation=True)

clinc_enc = clinc.map(tokenize_text, batched=True, remove_columns=["text"])
clinc_enc = clinc_enc.rename_column("intent", "labels")

Map:   0%|          | 0/15250 [00:00<?, ? examples/s]

Map:   0%|          | 0/3100 [00:00<?, ? examples/s]

Map:   0%|          | 0/5500 [00:00<?, ? examples/s]

In [17]:
accuracy_score = load_metric("accuracy")

def compute_metrics(pred):
 predictions, labels = pred
 predictions = np.argmax(predictions, axis=1)
 return accuracy_score.compute(predictions=predictions, references=labels)


In [18]:
batch_size = 48
finetuned_ckpt = "distilbert-base-uncased-finetuned-clinc"

student_training_args = DistillationTrainingArguments(
    output_dir=finetuned_ckpt, evaluation_strategy="epoch",
    num_train_epochs=5, learning_rate=2e-5,
    per_device_train_batch_size=batch_size,
    per_device_eval_batch_size=batch_size, alpha=1, weight_decay=0.01,
    push_to_hub=True)

In [19]:
id2label = pipe.model.config.id2label
label2id = pipe.model.config.label2id

In [20]:
num_labels = intents.num_classes
student_config = (AutoConfig
                  .from_pretrained(
                      student_ckpt,
                      num_labels=num_labels,
                      id2label=id2label, label2id=label2id
                      )
                  )

In [21]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

def student_init():
    return (AutoModelForSequenceClassification
            .from_pretrained(student_ckpt, config=student_config).to(device))


In [25]:
teacher_ckpt = "transformersbook/bert-base-uncased-finetuned-clinc"
teacher_model = (AutoModelForSequenceClassification
                 .from_pretrained(teacher_ckpt, num_labels=num_labels)
                 .to(device))

distilbert_trainer = DistillationTrainer(model_init=student_init,
                                         teacher_model=teacher_model,
                                         args=student_training_args,
                                         train_dataset=clinc_enc['train'],
                                         eval_dataset=clinc_enc['validation'],
                                         compute_metrics=compute_metrics,
                                         tokenizer=student_tokenizer)

distilbert_trainer.train()

loading configuration file config.json from cache at /root/.cache/huggingface/hub/models--transformersbook--bert-base-uncased-finetuned-clinc/snapshots/795b076da71dc236dde692338e21560cbbffa6e4/config.json
Model config BertConfig {
  "_name_or_path": "transformersbook/bert-base-uncased-finetuned-clinc",
  "architectures": [
    "BertForSequenceClassification"
  ],
  "attention_probs_dropout_prob": 0.1,
  "classifier_dropout": null,
  "gradient_checkpointing": false,
  "hidden_act": "gelu",
  "hidden_dropout_prob": 0.1,
  "hidden_size": 768,
  "id2label": {
    "0": "restaurant_reviews",
    "1": "nutrition_info",
    "2": "account_blocked",
    "3": "oil_change_how",
    "4": "time",
    "5": "weather",
    "6": "redeem_rewards",
    "7": "interest_rate",
    "8": "gas_type",
    "9": "accept_reservations",
    "10": "smart_home",
    "11": "user_name",
    "12": "report_lost_card",
    "13": "repeat",
    "14": "whisper_mode",
    "15": "what_are_your_hobbies",
    "16": "order",
    "

Epoch,Training Loss,Validation Loss,Accuracy
1,No log,3.288991,0.743226
2,3.786800,1.875629,0.837742
3,3.786800,1.157165,0.896129
4,1.692900,0.857340,0.913226
5,0.905800,0.772089,0.918387


***** Running Evaluation *****
  Num examples = 3100
  Batch size = 48
Saving model checkpoint to distilbert-base-uncased-finetuned-clinc/checkpoint-500
Configuration saved in distilbert-base-uncased-finetuned-clinc/checkpoint-500/config.json
Model weights saved in distilbert-base-uncased-finetuned-clinc/checkpoint-500/pytorch_model.bin
tokenizer config file saved in distilbert-base-uncased-finetuned-clinc/checkpoint-500/tokenizer_config.json
Special tokens file saved in distilbert-base-uncased-finetuned-clinc/checkpoint-500/special_tokens_map.json
tokenizer config file saved in distilbert-base-uncased-finetuned-clinc/tokenizer_config.json
Special tokens file saved in distilbert-base-uncased-finetuned-clinc/special_tokens_map.json
***** Running Evaluation *****
  Num examples = 3100
  Batch size = 48
***** Running Evaluation *****
  Num examples = 3100
  Batch size = 48
Saving model checkpoint to distilbert-base-uncased-finetuned-clinc/checkpoint-1000
Configuration saved in distilbert-

TrainOutput(global_step=1590, training_loss=2.0514509908808103, metrics={'train_runtime': 258.1391, 'train_samples_per_second': 295.383, 'train_steps_per_second': 6.159, 'total_flos': 413896353421488.0, 'train_loss': 2.0514509908808103, 'epoch': 5.0})

In [26]:
distilbert_trainer.push_to_hub("Training completed!")

Saving model checkpoint to distilbert-base-uncased-finetuned-clinc
Configuration saved in distilbert-base-uncased-finetuned-clinc/config.json
Model weights saved in distilbert-base-uncased-finetuned-clinc/pytorch_model.bin
tokenizer config file saved in distilbert-base-uncased-finetuned-clinc/tokenizer_config.json
Special tokens file saved in distilbert-base-uncased-finetuned-clinc/special_tokens_map.json
Several commits (2) will be pushed upstream.
The progress bars may be unreliable.


Upload file pytorch_model.bin:   0%|          | 32.0k/256M [00:00<?, ?B/s]

Upload file runs/Mar09_15-05-54_8664d4a1fe1d/events.out.tfevents.1678374658.8664d4a1fe1d.663.0: 100%|#########…

remote: Scanning LFS files of refs/heads/main for validity...        
remote: LFS file scan complete.        
To https://huggingface.co/pridaj/distilbert-base-uncased-finetuned-clinc
   de26e8e..73ac913  main -> main

remote: LFS file scan complete.        
To https://huggingface.co/pridaj/distilbert-base-uncased-finetuned-clinc
   de26e8e..73ac913  main -> main

To https://huggingface.co/pridaj/distilbert-base-uncased-finetuned-clinc
   73ac913..8156654  main -> main

   73ac913..8156654  main -> main



'https://huggingface.co/pridaj/distilbert-base-uncased-finetuned-clinc/commit/73ac913f7531414524d53ea9bae755b0122721c2'

In [27]:
finetuned_ckpt = "transformersbook/distilbert-base-uncased-finetuned-clinc"
pipe = pipeline("text-classification", model=finetuned_ckpt)

loading configuration file config.json from cache at /root/.cache/huggingface/hub/models--transformersbook--distilbert-base-uncased-finetuned-clinc/snapshots/0993da273a157b79a93c71901ed99fb71b861b02/config.json
Model config DistilBertConfig {
  "_name_or_path": "transformersbook/distilbert-base-uncased-finetuned-clinc",
  "activation": "gelu",
  "architectures": [
    "DistilBertForSequenceClassification"
  ],
  "attention_dropout": 0.1,
  "dim": 768,
  "dropout": 0.1,
  "hidden_dim": 3072,
  "id2label": {
    "0": "restaurant_reviews",
    "1": "nutrition_info",
    "2": "account_blocked",
    "3": "oil_change_how",
    "4": "time",
    "5": "weather",
    "6": "redeem_rewards",
    "7": "interest_rate",
    "8": "gas_type",
    "9": "accept_reservations",
    "10": "smart_home",
    "11": "user_name",
    "12": "report_lost_card",
    "13": "repeat",
    "14": "whisper_mode",
    "15": "what_are_your_hobbies",
    "16": "order",
    "17": "jump_start",
    "18": "schedule_meeting",
 

loading weights file pytorch_model.bin from cache at /root/.cache/huggingface/hub/models--transformersbook--distilbert-base-uncased-finetuned-clinc/snapshots/0993da273a157b79a93c71901ed99fb71b861b02/pytorch_model.bin
All model checkpoint weights were used when initializing DistilBertForSequenceClassification.

All the weights of DistilBertForSequenceClassification were initialized from the model checkpoint at transformersbook/distilbert-base-uncased-finetuned-clinc.
If your task is similar to the task the model of the checkpoint was trained on, you can already use DistilBertForSequenceClassification for predictions without further training.


loading file vocab.txt from cache at /root/.cache/huggingface/hub/models--transformersbook--distilbert-base-uncased-finetuned-clinc/snapshots/0993da273a157b79a93c71901ed99fb71b861b02/vocab.txt
loading file tokenizer.json from cache at /root/.cache/huggingface/hub/models--transformersbook--distilbert-base-uncased-finetuned-clinc/snapshots/0993da273a157b79a93c71901ed99fb71b861b02/tokenizer.json
loading file added_tokens.json from cache at None
loading file special_tokens_map.json from cache at /root/.cache/huggingface/hub/models--transformersbook--distilbert-base-uncased-finetuned-clinc/snapshots/0993da273a157b79a93c71901ed99fb71b861b02/special_tokens_map.json
loading file tokenizer_config.json from cache at /root/.cache/huggingface/hub/models--transformersbook--distilbert-base-uncased-finetuned-clinc/snapshots/0993da273a157b79a93c71901ed99fb71b861b02/tokenizer_config.json


In [28]:
optim_type = "DistilBERT"
pb = PerformanceBenchmark(pipe, clinc["test"], optim_type=optim_type)
perf_metrics.update(pb.run_benchmark())

Model size (MB) is 255.88
avg latency: 60.47 +/- 2.61
Accuracy on test set - 0.858
